# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
import syft as sy
import torch as th

W0731 21:56:14.040539 4622665152 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/miniconda3/envs/pri/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0731 21:56:14.047728 4622665152 deprecation_wrapper.py:119] From /miniconda3/envs/pri/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
hook = sy.TorchHook(th)

In [3]:
from torch import nn, optim

In [4]:
# Create a couple workers
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [5]:
# Make workers know each other
# bob.add_workers([alice, secure_worker])
# alice.add_workers([bob, secure_worker])
# secure_worker.add_workers([bob, alice])

In [6]:
# Same toy dataset for test
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]],requires_grad=True)

In [7]:
# get pointers to training data on each worker by 
# sending some training data to bob and alice

bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [8]:
# Initialize a toy model
model = nn.Linear(2,1)

In [9]:
model.weight.data.set_

<function Tensor.set_>

In [10]:
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [11]:
# optimizer sent to bob and alice
bob_optim = optim.SGD(params=bob_model.parameters(), lr=0.1)
alice_optim = optim.SGD(params=alice_model.parameters(), lr=0.1)

In [12]:
# Training

for i in range(20):
    bob_optim.zero_grad()
    bob_pred = bob_model(bob_data)
    bob_loss = ((bob_pred - bob_target)**2).sum()
    bob_loss.backward()
    bob_optim.step()
    bob_loss = bob_loss.get().data

#     print(bob_loss)

    alice_optim.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss = ((alice_pred - alice_target)**2).sum()
    alice_loss.backward()
    alice_optim.step()
    alice_loss = alice_loss.get().data

#     print(alice_loss)


In [13]:
bob_model.move(secure_worker)
alice_model.move(secure_worker)

In [14]:
secure_worker

<VirtualWorker id:secure_worker #objects:4>

In [15]:
secure_worker._objects

{9552132694: Parameter containing:
 tensor([[-0.1334,  0.0091]], requires_grad=True),
 77499361028: Parameter containing:
 tensor([-0.0056], requires_grad=True),
 68332878901: Parameter containing:
 tensor([[0.4528, 0.0493]], requires_grad=True),
 84388216758: Parameter containing:
 tensor([0.5195], requires_grad=True)}

In [16]:
(bob_model.weight.data + alice_model.weight.data)/2

(Wrapper)>[PointerTensor | me:10964742646 -> secure_worker:85103447583]

In [17]:
bob_model.weight.data

(Wrapper)>[PointerTensor | me:9552132694 -> secure_worker:9552132694]::data

In [18]:
alice_model.weight.data

(Wrapper)>[PointerTensor | me:68332878901 -> secure_worker:68332878901]::data

In [19]:
model.weight.data = ((bob_model.weight.data + alice_model.weight.data)/2).get()

In [20]:
bob_model.bias.data

(Wrapper)>[PointerTensor | me:77499361028 -> secure_worker:77499361028]::data

In [21]:
alice_model.bias.data

(Wrapper)>[PointerTensor | me:84388216758 -> secure_worker:84388216758]::data

In [22]:
model.bias.data = ((bob_model.bias.data + alice_model.bias.data)/2).get()

In [23]:
model.weight.data

tensor([[0.1597, 0.0292]])

In [24]:
model.bias.data

tensor([0.2569])

In [25]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [26]:
alice._objects

{42733477236: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 92874077483: tensor([[1.],
         [1.]], requires_grad=True), 96212792349: tensor([[0.9697],
         [1.0237]], grad_fn=<AddmmBackward>)}

In [27]:
bob._objects

{92539697895: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 61054651751: tensor([[0.],
         [0.]], requires_grad=True), 17083756746: tensor([[-0.0061],
         [ 0.0038]], grad_fn=<AddmmBackward>)}

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [28]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [29]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [30]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [31]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [32]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [33]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [34]:
import random

In [55]:
Q = 38748572002845748572040

In [56]:
x = 5

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [57]:
n_shares = 3
shares = list()

for i in range(n_shares-1):
    shares.append(random.randint(0,Q)) 
    print(shares)

[11714935009413870974269]
[11714935009413870974269, 22999942755410791649889]


In [58]:
sum(shares)

34714877764824662624158

In [60]:
sum(shares)%Q

34714877764824662624158

In [61]:
final_share = Q - (sum(shares)%Q)+x

In [62]:
final_share

4033694238021085947887

In [63]:
shares.append(final_share)

In [64]:
sum(shares)%Q

5

In [67]:
def encrypt(x, n_shares=3):
    shares = list()
    for i in range(n_shares-1):
        shares.append(random.randint(0,Q))
        
    final_share = Q - (sum(shares)%Q) + x
    shares.append(final_share)
    return tuple(shares)

In [68]:
encrypt(3, 5)

(14628715098078037621514,
 13089481483910584017592,
 8607548468159100472742,
 6208318629834253477515,
 34963080325709521554720)

In [69]:
def decrypt(shares):
    return sum(shares)%Q

In [71]:
decrypt(encrypt(5))

5

In [86]:
def add(a, b):
    
    c = list()
    assert(len(a)==len(b))
    print(len(a))
    
    for i in range(len(a)):
        c.append((a[i]+b[i])%Q)
        
    return tuple(c)
    

In [87]:
decrypt(add(encrypt(5), encrypt(7)))

3


12

In [88]:
x=encrypt(5)
x

(23124205286654065536284, 11616029789996745307317, 4008336926194937728444)

In [89]:
y=encrypt(7)
y

(9828424005154684530029, 979553862527634537399, 27940594135163429504619)

In [90]:
z = add(x, y)
z

3


(32952629291808750066313, 12595583652524379844716, 31948931061358367233063)

In [83]:
decrypt(z)

12

In [84]:
# Amazing 

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is `to aggregate gradients` using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [91]:
BASE=10
PRECISION=4

In [92]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [93]:
encode(3.5)

35000

In [94]:
decode(35000)

3.5

In [95]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

3


7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [108]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [109]:
x = th.tensor([1,2,3,4,5])

In [110]:
encrypt(1)

(24986125260173960709586, 4697792260958205682749, 9064654481713582179706)

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [111]:
x = x.share(bob, alice, secure_worker)

In [116]:
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:41791288661 -> bob:33457633693]
	-> (Wrapper)>[PointerTensor | me:31213652250 -> alice:65205632435]
	-> (Wrapper)>[PointerTensor | me:73109574446 -> secure_worker:15979179948]
	*crypto provider: me*

In [117]:
bob._objects

{33457633693: tensor([1129680468539563150,  680527768280160918,  576871781482202235,
         2281608821679165730, 2563380462999239835])}

In [118]:
alice._objects

{65205632435: tensor([ -111283587652916936,  3438017031814176307,  1774740197177325952,
           314180859908506459, -1297210351575912039])}

In [119]:
secure_worker._objects

{15979179948: tensor([-1018396880886646213, -4118544800094337223, -2351611978659528184,
         -2595789681587672185, -1266170111423327791])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [113]:
y = x + x

In [120]:
bob._objects

{33457633693: tensor([1129680468539563150,  680527768280160918,  576871781482202235,
         2281608821679165730, 2563380462999239835])}

In [114]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:67003562432 -> bob:42601721974]
	-> (Wrapper)>[PointerTensor | me:3435212361 -> alice:51618453603]
	-> (Wrapper)>[PointerTensor | me:59527101788 -> secure_worker:61757932156]
	*crypto provider: me*

In [115]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [125]:
x = th.tensor([0.1,0.2,0.3])

In [126]:
x

tensor([0.1000, 0.2000, 0.3000])

In [127]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [130]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [132]:
x.child

FixedPrecisionTensor>tensor([100, 200, 300])

In [133]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [128]:
x.child.child

tensor([100, 200, 300])

In [129]:
type(x.child.child)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [134]:
y = x + x

In [135]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [136]:
x = th.tensor([0.1, 0.2, 0.3])

In [137]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [138]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:60173721446 -> bob:28297185475]
	-> (Wrapper)>[PointerTensor | me:53086570343 -> alice:30161380170]
	-> (Wrapper)>[PointerTensor | me:84637756551 -> secure_worker:26148452594]
	*crypto provider: me*

In [139]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:96377106476 -> bob:84508062613]
	-> (Wrapper)>[PointerTensor | me:54898402457 -> alice:25548401828]
	-> (Wrapper)>[PointerTensor | me:19961289908 -> secure_worker:87958278998]
	*crypto provider: me*

In [140]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation